<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-setting" data-toc-modified-id="Data-setting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data setting</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Response" data-toc-modified-id="Response-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Response</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Test</a></span></li><li><span><a href="#Appendix" data-toc-modified-id="Appendix-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Appendix</a></span></li></ul></div>

In [1]:
import pandas as pd

import torch
import mymodel
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable 
import pickle 

import MeCab
from mecab_split import mecabsplit
tagger = MeCab.Tagger()

## Data setting

In [2]:
df=pd.read_csv('college_faq_data.csv',encoding='utf-8')

In [3]:
df.head()

,category,question,answer
0,등록,대학원 수료생(당 학기 논문제출 예정자)의 경우 등록금 및 생활비 대출이 가능한가요?,대학원 수료생의 경우 일부 금액이 등록금 개념으로 고지된다 하더라도 이 비용은 학점...
1,등록,비학위과정의 대학원생은 등록금 및 생활비 대출이 가능한가요?,학점 인정을 떠나 비학위과정의 재학생은 대출자격요건에 충족되지 않습니다.\r\n\r...
2,등록,"4년제 대학의 9학기, 2년제의 5학기 등과 같은 학기초과자 학생도 학자금 대출을 ...",원칙적으로 취업을 위한 의도적 졸업유예의 경우 학자금 대출이 불가합니다. 다만 졸업...
3,등록,학생 평점이 백분위 환산점수로 79.7점입니다. 든든학자금 대출이 가능한가요?,든든학자금 대출의 성적 기준은 80점 이상입니다. 성적은 반올림 처리가 되지 않으므...
4,등록,"재입학생이나 편입생은 재학생군인가요, 신입생 군인가요?",재입학생과 편입생 모두 신입생군에 속합니다.\r\n\r\n*부서 : 재무회계팀\r\...


In [4]:
df.category.value_counts()

등록           73
생활관          23
증명서          23
휴복학          20
복수전공/연계전공    20
수강신청         20
기타           17
계절제수업        17
장학금          15
재수강          14
캠퍼스견학        12
성적/출결        12
졸업           11
소속변경         10
도서관          10
ID카드          9
건강센터          9
YSCEC         8
교환학생          7
자원봉사          4
성폭력/성평등       3
장애학생지원        2
교원자격증         1
Name: category, dtype: int64

In [5]:
subset_category=['등록','생활관','수강신청','휴복학','장학금']

df=df[df.category.isin(subset_category)]

In [6]:
content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in df.category.unique():
    intent=line
    if intent not in intent_vocab:
        intent_vocab[intent]=intent_idx
        intent_list.append(intent)
        intent_idx +=1

for line in list(df.question):
    line = mecabsplit(tagger,line,False)
    for it in line:
        if it not in content_vocab:
            content_vocab[it] = vocab_idx # it : 단어
            vocab_idx +=1

## Modeling

In [7]:
cnn = mymodel.CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(클래스)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()    # train mode로 준비

451 5


CNN_Text(
  (embed): Embedding(451, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(1, 100), stride=(1, 1))
    (1): Conv2d(1, 20, kernel_size=(2, 100), stride=(1, 1))
    (2): Conv2d(1, 20, kernel_size=(3, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2)
  (fc1): Linear(in_features=60, out_features=5, bias=True)
)

## Training

In [8]:
epoch = 10
for e in range(epoch):
    totalloss = 0
    
    for line in df.values:
        target=Variable(torch.LongTensor([intent_vocab[line[0]]]))

        optimizer.zero_grad()

        cont = []
        line = mecabsplit(tagger,line[1],False)
        for it in line:
            cont.append(content_vocab[it])
        
        
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)
        
        loss = F.cross_entropy(pred,target)
        totalloss += loss.data[0]
        loss.backward()  # 필터, 임베딩벡터 값, FC 레이어의 matrix 등 모든 값을 loss를 줄이는 방향으로 계산
        optimizer.step() # 벡터값을 변경함
    print (totalloss)

242.97551809606375
169.37900107493624
96.56319632707164
57.140459277143236
35.68843705614563
18.125835393228044
12.299512944482558
10.355155608986024
6.935198462255357
3.892052376792435


## Response

In [9]:
response=list(df.answer)

## Test

In [10]:
test=[]
for i in intent_list:
    subset_df=df.query('category=="{}"'.format(i))
    test.append(subset_df.question.values[0])

In [11]:
test

['대학원 수료생(당 학기 논문제출 예정자)의 경우 등록금 및 생활비 대출이 가능한가요?',
 '동록금을 납부한 뒤 휴학을 하면 나중에 복학할 때 등록금을 납부하지 않아도 되나요?',
 '2, 3차 복학 후 수강신청시 불이익이 있나요?',
 '한국장학재단 학자금 대출 시 특별 추천은 어떤 경우에 받을 수 있나요?',
 '생활관과 신촌역을 연결하는 교통편이 있나요?']

In [12]:
## your test input
test=['1학기 수강신청 기간 언제인가요?']

In [13]:
for idx in list(test):
    line = mecabsplit(tagger,idx,False)
   
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it])  # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    v,i = torch.max(pred,1) # pred는 (p1, p2) 즉, 클래스별 확률 v: 둘중 큰값 i:큰값 클래스의 인덱스
    
    print('input : ',line)
    print('intent : ',intent_list[int(i)])
    print('answer : ',response[int(i)],'\n')

input :  ['1', '학기', '수강', '신청', '기간', '언제', '이', 'ᆫ가요', '?']
intent :  수강신청
answer :  원칙적으로 취업을 위한 의도적 졸업유예의 경우 학자금 대출이 불가합니다. 다만 졸업유예가 아닌 학교에서 인정하는 정규학기일 경우 학사정보 및 수납원장을 해당 졸업학년으로 업로드 해 주시면 학자금 대출이 가능합니다.

*부서 : 재무회계팀
*전화번호 : 02-2123-4500 

